# Introduction to Scattertext

## @jasonkessler

https://github.com/JasonKessler/scattertext



Cite as:
Jason S. Kessler. Scattertext: a Browser-Based Tool for Visualizing how Corpora Differ. Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (ACL): System Demonstrations. 2017.

Link to preprint: https://arxiv.org/abs/1703.00565

`
@article{kessler2017scattertext,
  author    = {Kessler, Jason S.},
  title     = {Scattertext: a Browser-Based Tool for Visualizing how Corpora Differ},
  booktitle = {ACL System Demonstrations},
  year      = {2017},
}
`

In [1]:
# pip install scattertext agefromname

In [2]:
# pip install spacy

In [3]:
%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))

In [6]:
# nlp = spacy.load('en')
# If this doesn't work, please uncomment the following line and use a regex-based parser instead
nlp = st.whitespace_nlp_with_sentences

# Grab the 2012 political convention data set and preview it

In [7]:
convention_df = st.SampleCorpora.ConventionData2012.get_data()

In [8]:
convention_df.iloc[0]

party                                               democrat
text       Thank you. Thank you. Thank you. Thank you so ...
speaker                                         BARACK OBAMA
Name: 0, dtype: object

In [9]:
print("Document Count")
print(convention_df.groupby('party')['text'].count())
print("Word Count")
convention_df.groupby('party').apply(lambda x: x.text.apply(lambda x: len(x.split())).sum())
convention_df['parsed'] = convention_df.text.apply(nlp)

Document Count
party
democrat      123
republican     66
Name: text, dtype: int64
Word Count


# Turn it into a Scattertext corpus, and have spaCy parse it.

In [10]:
corpus = st.CorpusFromParsedDocuments(convention_df, category_col='party', parsed_col='parsed').build()

# Scattertext has some functions to find how associated words are with categories 
# I've reworded this section since the talk
## Lots of ways to do this. I'm partial to a novel technique called Scaled F-Score
# Intutition:
### Associatied terms have a *relatively* high category-specific precision and category-specific term frequency (i.e., % of terms in category are term)
### Take the harmonic mean of precision and frequency (both have to be high)
### Hyper-parameters are pretty much universal (beta and transformation function)

Given a word $w_i \in W$ and a category $c_j \in C$, define the precision of the word $w_i$ wrt to a category as:
$$ \mbox{prec}(w_i, c_j) = \frac{\#(w_i, c_j)}{\sum_{c \in C} \#(w_i, c)}. $$

The function $\#(w_i, c_j)$ represents either the number of times $w_i$ occurs in a document labeled with the category $c_j$ or the number of documents labeled $c_j$ which contain $w_i$.

Similarly, define the frequency a word occurs in the category as:

$$ \mbox{freq}(w_i, c_j) = \frac{\#(w_i, c_j)}{\sum_{w \in W} \#(w, c_j)}. $$

The F-Score of these two values is defined as:

$$ \mathcal{F}_\beta(\mbox{prec}, \mbox{freq}) = (1 + \beta^2) \frac{\mbox{prec} \cdot \mbox{freq}}{\beta^2 \cdot \mbox{prec} + \mbox{freq}}. $$

$\beta \in \mathcal{R}^+$ is a scaling factor where frequency is favored if $\beta < 1$, precision if $\beta > 1$, and both are equally weighted if $\beta = 1$. F-Score is equivalent to the harmonic mean where $\beta = 1$.

In [11]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['dem_precision'] = term_freq_df['democrat freq'] * 1./(term_freq_df['democrat freq'] + term_freq_df['republican freq'])
term_freq_df['dem_freq_pct'] = term_freq_df['democrat freq'] * 1./term_freq_df['democrat freq'].sum()
term_freq_df['dem_hmean'] = term_freq_df.apply(lambda x: (hmean([x['dem_precision'], x['dem_freq_pct']])
                                                                   if x['dem_precision'] > 0 and x['dem_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df.sort_values(by='dem_hmean', ascending=False).iloc[:10]

,democrat freq,republican freq,dem_precision,dem_freq_pct,dem_hmean
term,,,,,
the,3400,2534,0.572969,0.022364,0.043049
and,2704,2236,0.547368,0.017786,0.034453
to,2341,1668,0.583936,0.015399,0.030006
a,1602,1349,0.542867,0.010538,0.020674
of,1568,1379,0.532067,0.010314,0.020236
that,1400,1052,0.570962,0.009209,0.018125
we,1320,1146,0.535280,0.008683,0.017088
in,1294,988,0.567046,0.008512,0.016772
i,1105,858,0.562914,0.007268,0.014352


## Solution:
### Take the normal CDF of precision and frequency percentage scores, which will fall between 0 and 1, which scales and standardizes both scores.

Define the the Normal CDF as:

$$ \Phi(z) = \int_{-\infty}^z \mathcal{N}(x; \mu, \sigma^2)\ \mathrm{d}x.$$

Where $ \mathcal{N} $ is the PDF of the Normal distribution, $\mu$ is the mean, and $\sigma^2$ is the variance.

$\Phi$ is used to scale and standardize the precisions and frequencies, and place them on the same scale $[0,1]$.

Now we can define Scaled F-Score as the harmonic mean of the Normal CDF transformed frequency and precision:

$$ \mathcal{S}_{\beta}(w_i, c_j) = \mathcal{F}_{\beta}(\Phi(\mbox{prec}(w_i, c_j)), \Phi(\mbox{freq}(w_i, c_j))).$$

$\mu$ and $\sigma^2$ are defined separately as the mean and variance of precision and frequency.

A $\beta$ of 0.5 is recommended and is the default value in Scattertext.

Note that any function with the range of $[0,1]$ (this includes the identity function) may be used in place of $\Phi$.

In [12]:
def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())
term_freq_df['dem_precision_normcdf'] = normcdf(term_freq_df['dem_precision'])
term_freq_df['dem_freq_pct_normcdf'] = normcdf(term_freq_df['dem_freq_pct'])
term_freq_df['dem_scaled_f_score'] = hmean([term_freq_df['dem_precision_normcdf'], term_freq_df['dem_freq_pct_normcdf']])
term_freq_df.sort_values(by='dem_scaled_f_score', ascending=False).iloc[:10]

,democrat freq,republican freq,dem_precision,dem_freq_pct,dem_hmean,dem_precision_normcdf,dem_freq_pct_normcdf,dem_scaled_f_score
term,,,,,,,,
middle class,152,19,0.888889,0.001000,0.001997,0.768254,1.000000,0.868941
auto,37,0,1.000000,0.000243,0.000487,0.836306,0.888460,0.861594
fair,45,3,0.937500,0.000296,0.000592,0.799786,0.933269,0.861387
insurance,54,6,0.900000,0.000355,0.000710,0.775696,0.965474,0.860242
forward,106,16,0.868852,0.000697,0.001393,0.754494,0.999870,0.860022
president barack,47,4,0.921569,0.000309,0.000618,0.789747,0.941925,0.859149
class,161,25,0.865591,0.001059,0.002115,0.752213,1.000000,0.858587
middle,168,28,0.857143,0.001105,0.002207,0.746253,1.000000,0.854691
the middle,98,17,0.852174,0.000645,0.001288,0.742713,0.999624,0.852228


In [13]:
term_freq_df['dem_corner_score'] = corpus.get_corner_scores('democrat')
term_freq_df.sort_values(by='dem_corner_score', ascending=False).iloc[:10]

,democrat freq,republican freq,dem_precision,dem_freq_pct,dem_hmean,dem_precision_normcdf,dem_freq_pct_normcdf,dem_scaled_f_score,dem_corner_score
term,,,,,,,,,
auto,37,0,1.0,0.000243,0.000487,0.836306,0.888460,0.861594,0.919662
america forward,28,0,1.0,0.000184,0.000368,0.836306,0.816177,0.826119,0.919631
auto industry,24,0,1.0,0.000158,0.000316,0.836306,0.776309,0.805191,0.919607
insurance companies,24,0,1.0,0.000158,0.000316,0.836306,0.776309,0.805191,0.919607
pell,23,0,1.0,0.000151,0.000303,0.836306,0.765624,0.799405,0.919597
last week,22,0,1.0,0.000145,0.000289,0.836306,0.754662,0.793389,0.919587
pell grants,21,0,1.0,0.000138,0.000276,0.836306,0.743431,0.787138,0.919574
women s,20,0,1.0,0.000132,0.000263,0.836306,0.731937,0.780648,0.919561
platform,20,0,1.0,0.000132,0.000263,0.836306,0.731937,0.780648,0.919561


In [14]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['Republican Score'] = corpus.get_scaled_f_scores('republican')
term_freq_df['Democratic Score'] = corpus.get_scaled_f_scores('democrat')
print("Top 10 Democratic terms")
pprint(list(term_freq_df.sort_values(by='Democratic Score', ascending=False).index[:10]))
print("Top 10 Republican terms")
pprint(list(term_freq_df.sort_values(by='Republican Score', ascending=False).index[:10]))

Top 10 Democratic terms
['middle class',
 'forward',
 'class',
 'middle',
 'the middle',
 'pay',
 'medicare',
 'education',
 'health',
 'president obama']
Top 10 Republican terms
['government',
 'administration',
 'business',
 'success',
 'can do',
 'story',
 'unemployment',
 'freedom',
 'paul',
 'do better']


# Make and visualize chart, scale based on raw frequency.
### - A word used 10 times by Republicans will be at position 10 on the on the x-axis 
### - This isn't very useful.  Everything but the most frequent terms are squished the lower-left corner
### - The corner-distance scores are largely stopwords
### - By default, color words by Scaled F-Score

In [23]:
html = produce_scattertext_explorer(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.scale,
                                    metadata=convention_df['speaker'])
file_name = 'output/Conventions2012ScattertextScale.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)


TypeError: scale() takes from 1 to 2 positional arguments but 3 were given

## Using log scales seems to help a bit, but blank space and stop words still dominate the graph
### The chracteristic terms look much more informative

In [24]:
html = st.produce_scattertext_explorer(corpus,
                                       category='democrat',
                                       category_name='Democratic',
                                       not_category_name='Republican',
                                       minimum_term_frequency=5,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'output/Conventions2012ScattertextLog.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)


# Rank terms by frequency percentiles instead of raw frequenies.  
### A term at the middle of the x-axis will be mentioned by Republicans at the median frequency.
### This nicely distributes terms throughout the space
### But, terms occuring with the same frequencies in both classes are stacked atop each other.
### Can't mouseover points not at top of stack.

In [26]:
html = produce_scattertext_explorer(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.percentile,
                                    metadata=convention_df['speaker'])
file_name = 'output/Conventions2012ScattertextRankData.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)


# One solution is to randomly jitter each point
## Points don't leave enough space for many labels
## Top terms laregely result of jitter

In [22]:
html = produce_scattertext_explorer(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    width_in_pixels=1000,
                                    jitter=0.1,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.percentile,
                                    metadata=convention_df['speaker'])
file_name = 'output/Conventions2012ScattertextRankDataJitter.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)


# The preferred solution is to fall back to alphabetic order among equally frequent terms
## Lets you mouseover all points
## Leaves a bit of room for labels
## Top points may be slightly distorted

In [25]:
html = produce_scattertext_explorer(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    metadata=convention_df['speaker'],
                                    term_significance = st.LogOddsRatioUninformativeDirichletPrior())
file_name = 'output/Conventions2012ScattertextRankDefault.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

# Scattertext can also be used for alternative visualizations
## Visualize L2-penalized logistic regression coefficients vs. log term frequency

In [27]:
def scale(ar): 
    return (ar - ar.min()) / (ar.max() - ar.min())

def zero_centered_scale(ar):
    scores = np.zeros(len(ar))
    scores[ar > 0] = scale(ar[ar > 0])
    scores[ar < 0] = -scale(-ar[ar < 0])
    return (scores + 1) / 2.

frequencies_scaled = scale(np.log(term_freq_df.sum(axis=1).values))

In [28]:
from sklearn.linear_model import LogisticRegression
scores = corpus.get_logreg_coefs('democrat',
                                 LogisticRegression(penalty='l2', C=10, max_iter=10000, n_jobs=-1))
scores_scaled = zero_centered_scale(scores)

html = produce_scattertext_explorer(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    minimum_term_frequency=5,
                                    width_in_pixels=1000,
                                    x_coords=frequencies_scaled,
                                    y_coords=scores_scaled,
                                    scores=scores,
                                    sort_by_dist=False,
                                    metadata=convention_df['speaker'],
                                    x_label='Log frequency',
                                    y_label='L2-Penalized Log Reg Coef')
file_name = 'output/L2vsLog.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

## We can see how this compares to Scaled F-Score

In [29]:
html = produce_scattertext_explorer(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    minimum_term_frequency=5,
                                    width_in_pixels=1000,
                                    x_coords=frequencies_scaled,
                                    y_coords=corpus.get_scaled_f_scores('democrat', beta=0.5),
                                    scores=corpus.get_scaled_f_scores('democrat', beta=0.5),
                                    sort_by_dist=False,
                                    metadata=convention_df['speaker'],
                                    x_label='Log Frequency',
                                    y_label='Scaled F-Score')
file_name = 'output/SFSvsLog.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

# Penalized log-odds-ratio has recently become popular recently

Burt L. Monroe, Michael P. Colaresi, and Kevin M. Quinn. 2008. Fightin’ words: Lexical feature selection and evaluation for identifying the content of political conflict. Political Analysis.

In [30]:
freq_df = corpus.get_term_freq_df().rename(columns={'democrat freq': 'y_dem', 'republican freq': 'y_rep'})
a_w = 0.01
y_i, y_j = freq_df['y_dem'].values, freq_df['y_rep'].values

In [31]:
n_i, n_j = y_i.sum(), y_j.sum()
a_0 = len(freq_df) * a_w
delta_i_j = (  np.log((y_i + a_w) / (n_i + a_0 - y_i - a_w))
                 - np.log((y_j + a_w) / (n_j + a_0 - y_j - a_w)))
var_delta_i_j = ( 1./(y_i + a_w) + 1./(y_i + a_0 - y_i - a_w)
                    + 1./(y_j + a_w) + 1./(n_j + a_0 - n_j - a_w))
zeta_i_j = delta_i_j/np.sqrt(var_delta_i_j)
max_abs_zeta = max(zeta_i_j.max(), -zeta_i_j.min())
zeta_scaled_for_charting = ((((zeta_i_j > 0).astype(float) * (zeta_i_j/max_abs_zeta))*0.5 + 0.5)
                            + ((zeta_i_j < 0).astype(float) * (zeta_i_j/max_abs_zeta) * 0.5))

In [32]:
html = produce_scattertext_explorer(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    minimum_term_frequency=5,
                                    width_in_pixels=1000,
                                    x_coords=frequencies_scaled,
                                    y_coords=zeta_scaled_for_charting,
                                    scores=zeta_i_j,
                                    sort_by_dist=False,
                                    metadata=convention_df['speaker'],
                                    x_label='Log Frequency',
                                    y_label='Log Odds Ratio w/ Uninformative Prior (alpha_w=0.01)')
file_name = 'output/LOPriorvsLog.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

## And finally, corner score

In [33]:
corner_scores = corpus.get_corner_scores('democrat')
html = produce_scattertext_explorer(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    minimum_term_frequency=5,
                                    width_in_pixels=1000,
                                    x_coords=frequencies_scaled,
                                    y_coords=corner_scores,
                                    scores=corner_scores,
                                    sort_by_dist=False,
                                    metadata=convention_df['speaker'],
                                    x_label='Log Frequency',
                                    y_label='Corner Scores')
file_name = 'output/CornervsLog.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)